In [ ]:
import time
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import os

In [ ]:
from datetime import datetime

In [ ]:
from pandas.io import sql
from sqlalchemy import create_engine
import sqlalchemy
import mysql.connector

In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
from unidecode import unidecode

In [ ]:
path_notebook = os.getcwd()

In [ ]:
#Inicio da execução do notebook
start_general = datetime.now()

## Definir funções

In [ ]:
def cadastrar_fonte_dados (nome_site, site_fonte_dados, nome_fonte_dados):
    
    fonte_dados = [[nome_site, site_fonte_dados, nome_fonte_dados]]

    colunas = ['Nome_Site', 'Site_Fonte_Dados', 'Nome_Fonte_Dados']

    fonte_dados = pd.DataFrame(fonte_dados, columns = colunas)

    with engine.connect() as conn, conn.begin():
        fonte_dados.to_sql('stg_Fonte_Dados', conn, if_exists='replace', index=False)


    ## Executar procedure versionamento

    conexao = engine.connect()

    consulta_sql = "CALL Stage.Versiona_stg_Fonte_Dados_Internet();"

    with engine.begin() as conn:
        conn.execute(consulta_sql)


    ## Deletar dados tabela stage

    consulta_sql = """
        DELETE FROM Stage.stg_Fonte_Dados
        WHERE TRUE;
    """

    with engine.begin() as conn:
        conn.execute(consulta_sql)

    conexao.close()

    ### Identificar ID Fonte de dados

    conn = engine.connect()

    consulta_sql = """
        SELECT id_Fonte_Dados
        FROM Internet_db.Fonte_Dados_Internet
        WHERE Site_Fonte_Dados = '""" +  url  + """'

    """

    with engine.begin() as conn:
        result_query = conn.execute(consulta_sql).fetchall()

    return result_query[0][0]

In [ ]:
def consultar_fonte_dados(nome_site, site_fonte_dados, nome_fonte_dados):
    
    conn = engine.connect()

    consulta_sql = """
        SELECT id_Fonte_Dados
        FROM Internet_db.Fonte_Dados_Internet
        WHERE Site_Fonte_Dados = '""" +  site_fonte_dados  + """'

    """

    with engine.begin() as conn:
        result_query = conn.execute(consulta_sql).fetchall()


    if len(result_query) != 0:
        return result_query[0][0]

    else:
        print('Fonte de dados não está previamente cadastrada')
        print('Fonte de dados será cadastrada agora')

        return cadastrar_fonte_dados (nome_site, site_fonte_dados, nome_fonte_dados)

In [ ]:
def inserir_dados_stg(nome_data_frame, nome_banco_tabela, nome_tabela, nome_banco_procedure, nome_procedure):

    # Inserir dados tabela
    with engine.connect() as conn, conn.begin():
        nome_data_frame.to_sql(nome_tabela, conn, if_exists='replace', index=False)



    ## Executar procedure versionamento
    conexao = engine.connect()

    consulta_sql = "CALL "  + nome_banco_procedure + "." + nome_procedure + "();"

    with engine.begin() as conn:
        conn.execute(consulta_sql)



    ## Deletar dados tabela

    consulta_sql = """
        DELETE FROM """ + nome_banco_tabela + """.""" + nome_tabela + """
        WHERE TRUE;
    """

    with engine.begin() as conn:
        conn.execute(consulta_sql)

    conexao.close()

In [ ]:
def de_para_nome_cidade(nome_data_frame, sigla_estado, de_cidade, para_cidade, observacao):
    
    nome_data_frame['Observacao'] = np.where(
        (nome_data_frame['sigla_Estado'] == sigla_estado) & (nome_data_frame['Cidade_Join'] == de_cidade )
        ,observacao
        ,nome_data_frame['Observacao']
    )

    nome_data_frame['Cidade_Join'] = np.where(
        (nome_data_frame['sigla_Estado'] == sigla_estado) & (nome_data_frame['Cidade_Join'] == de_cidade )
        ,para_cidade
        ,nome_data_frame['Cidade_Join']
    )
    
    

## Criar conexão banco dados

In [ ]:
#texto = open('/root/airflow/scripts/config/conexao_db.txt')
texto = open('conexao_db.txt')
conexao_db = texto.read()
texto.close()

In [ ]:
engine = create_engine(conexao_db, encoding='utf-8')

In [ ]:
print(engine.table_names())

### Datas de vigências

In [ ]:
data_inicio_vigencia = datetime.now().strftime('%Y-%m-%d')

In [ ]:
data_fim_vigencia = '9999-12-31'

# Importar base de dados de países

In [ ]:
session_requests = requests.session()

In [ ]:
url = 'https://www.portalpower.com.br/lista-ddd-brasil-todos-ddds/'
consulta = session_requests.get(url)
data_extracao = datetime.now()

In [ ]:
consulta.ok # Will tell us if the last request was ok
consulta.status_code # Will give us the status from the last request

### Identificar ID Fonte de dados

In [ ]:
nome_site = 'portalpower'
site_fonte_dados = url
nome_fonte_dados = 'ddd-cidades-do-brasil'

In [ ]:
id_fonte_dados = consultar_fonte_dados(nome_site, site_fonte_dados, nome_fonte_dados)

In [ ]:
print('id_fonte_dados:' , id_fonte_dados)

## Conversões

In [ ]:
soup = BeautifulSoup(consulta.content, 'html.parser')

In [ ]:
ddd0 = pd.read_html(soup.prettify())

In [ ]:
estados = soup.find_all('h4')

In [ ]:
estados_ajustados = []

for j in range(0, len(estados), 1):
    if len(estados[j].getText()) > 0:
        estados_ajustados.append(estados[j].getText())

In [ ]:
quantidade_tabelas = len(ddd0)
print('quantidade_tabelas', quantidade_tabelas)

In [ ]:
try:
    del ddd
except:
    pass
ddd = pd.DataFrame(columns = {'Cidade', 'DDD', 'Estado'})

In [ ]:
for i in range(0, quantidade_tabelas -1, 1):
    temp = pd.DataFrame(columns = {'Cidade', 'DDD', 'Estado'})
    temp['Cidade'] = ddd0[i][0]
    temp['DDD'] = ddd0[i][1]
    temp['Estado'] = estados_ajustados[i]
    
    ddd = ddd.append(pd.DataFrame(data = temp), ignore_index=True)
    
    del temp

In [ ]:
ddd['sigla_Estado'] = ddd['Estado'].str[0:2]
ddd['Estado'] = ddd['Estado'].str[5:]

In [ ]:
ddd['Data_Extracao'] = data_extracao
ddd['id_Fonte_Dados'] = id_fonte_dados
ddd['Data_Inicio_Vigencia'] = data_inicio_vigencia
ddd['Data_Fim_Vigencia'] = data_fim_vigencia

### Consultar municípios

In [ ]:
### Identificar ID Fonte de dados
consulta_sql = """

    SELECT 
        mi.id_Municipio as id_Municipio_IBGE
        ,mi.nome_Municipio
        ,UPPER(mi.nome_Municipio) as Cidade_Join
        ,ei.id_Estado as id_Estado_IBGE
        ,ei.sigla_Estado 
    FROM
        IBGE.Municipio_IBGE as mi 
            inner join
        IBGE.Estado_IBGE as ei on (mi.id_Estado = ei.id_Estado)
    WHERE TRUE
        AND mi.Data_Fim_Vigencia = '9999-12-31'
        AND ei.Data_Fim_Vigencia = '9999-12-31'
"""

#with engine.begin() as conn:
#    result_query = conn.execute(consulta_sql, head=True).fetchall()


In [ ]:
municipios = pd.read_sql_query(consulta_sql, con=engine.connect())

In [ ]:
# Ajuste nome cidade dataframe municipios
# retirar acentos
municipios['Cidade_Join'] = list(map(lambda x: unidecode(x), list(municipios.Cidade_Join)))
municipios['Cidade_Join'] = municipios['Cidade_Join'].str.replace(" DO ", " DE ")
municipios['Cidade_Join'] = municipios['Cidade_Join'].str.replace(" DOS ", " DE ")
municipios['Cidade_Join'] = municipios['Cidade_Join'].str.replace(" DA ", " DE ")
municipios['Cidade_Join'] = municipios['Cidade_Join'].str.replace(" DAS ", " DE ")

In [ ]:
# Ajuste nome cidade dataframe DDD
ddd['Cidade_Join'] = ddd['Cidade'].str.replace("’", "'")
ddd['Cidade_Join'] = ddd['Cidade_Join'].str.replace(" DO ", " DE ")
ddd['Cidade_Join'] = ddd['Cidade_Join'].str.replace(" DOS ", " DE ")
ddd['Cidade_Join'] = ddd['Cidade_Join'].str.replace(" DA ", " DE ")
ddd['Cidade_Join'] = ddd['Cidade_Join'].str.replace(" DAS ", " DE ")

In [ ]:
ddd['Observacao'] = ''

### Auste de alteração de nomes de municipios

In [ ]:
de_para_nome_cidade(ddd, 'BA', 'GOVERNADOR LOMANTO JUNIOR', 'BARRO PRETO', 'https://pt.wikipedia.org/wiki/Barro_Preto')

de_para_nome_cidade(ddd, 'PB', 'CAMPO DE SANTANA', 'TACIMA', 'https://cidades.ibge.gov.br/brasil/pb/tacima/historico')

de_para_nome_cidade(ddd, 'PB', 'SERIDO', 'SAO VICENTE DE SERIDO', 'https://cidades.ibge.gov.br/brasil/pb/sao-vicente-do-serido/historico')

de_para_nome_cidade(ddd, 'RN', 'AUGUSTO SEVERO', 'CAMPO GRANDE', 'https://pt.wikipedia.org/wiki/Campo_Grande_(Rio_Grande_do_Norte)')

de_para_nome_cidade(ddd, 'RN', 'PRESIDENTE JUSCELINO', 'SERRA CAIADA', 'https://pt.wikipedia.org/wiki/Serra_Caiada')

de_para_nome_cidade(ddd, 'SC', 'PICARRAS', 'BALNEARIO PICARRAS', 'https://pt.wikipedia.org/wiki/Balne%C3%A1rio_Pi%C3%A7arras')

de_para_nome_cidade(ddd, 'SP', 'EMBU', 'EMBU DE ARTES', 'https://pt.wikipedia.org/wiki/Embu_das_Artes')

de_para_nome_cidade(ddd, 'PB', 'SAO DOMINGOS DE POMBAL', 'SAO DOMINGOS', 'Pendente validar fonte informação')

In [ ]:
# de_para_nome_cidade(ddd, '', '', '', '')

### Ajuste nome cidades, letras, etc

In [ ]:
de_para_nome_cidade(ddd, 'BA', 'SANTA TERESINHA', 'SANTA TEREZINHA', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'CE', 'ITAPAGE', 'ITAPAJE', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'MG', 'BRASOPOLIS', 'BRAZOPOLIS', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'MG', 'DONA EUSEBIA', 'DONA EUZEBIA', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'MG', 'ITABIRINHA DE MANTENA', 'ITABIRINHA', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'MG', 'PASSA-VINTE', 'PASSA VINTE', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'MG', 'SAO THOME DE LETRAS', 'SAO TOME DE LETRAS', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'MT', 'POXOREO', 'POXOREU', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'PA', 'SANTA ISABEL DE PARA', 'SANTA IZABEL DE PARA', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'PE', 'IGUARACI', 'IGUARACY', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'RJ', 'PARATI', 'PARATY', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'RJ', 'TRAJANO DE MORAIS', 'TRAJANO DE MORAES', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'RN', 'OLHO-D\'AGUA DE BORGES', 'OLHO D\'AGUA DE BORGES', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'RS', 'SANTANA DE LIVRAMENTO', 'SANT\'ANA DE LIVRAMENTO', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'SC', 'GRAO PARA', 'GRAO-PARA', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'SC', 'PRESIDENTE CASTELO BRANCO', 'PRESIDENTE CASTELLO BRANCO', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'SP', 'BIRITIBA-MIRIM', 'BIRITIBA MIRIM', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'SP', 'FLORINIA', 'FLORINEA', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'SP', 'MOJI DE CRUZES', 'MOGI DE CRUZES', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'SP', 'MOJI-MIRIM', 'MOGI MIRIM', 'Ajuste letra nome')

de_para_nome_cidade(ddd, 'SP', 'SAO LUIS DE PARAITINGA', 'SAO LUIZ DE PARAITINGA', 'Ajuste letra nome')

In [ ]:
#de_para_nome_cidade(ddd, '', '', '', 'Ajuste letra nome')

In [ ]:
ddd = pd.merge(
    ddd
    ,municipios[['sigla_Estado', 'Cidade_Join', 'id_Municipio_IBGE', 'id_Estado_IBGE', 'nome_Municipio']]
    ,left_on = ('sigla_Estado', 'Cidade_Join')
    ,right_on = ('sigla_Estado', 'Cidade_Join')
    ,how = 'left'
)

In [ ]:
len(ddd[(ddd['id_Municipio_IBGE'].isnull())])

In [ ]:
# Selecionar colunas
ddd = ddd[[
    'id_Fonte_Dados'
    ,'id_Estado_IBGE'
    ,'id_Municipio_IBGE'
    ,'nome_Municipio'
    ,'Cidade'
    ,'DDD'
    ,'Observacao'
    ,'Data_Extracao'
    ,'Data_Inicio_Vigencia'
    ,'Data_Fim_Vigencia'    
]]

In [ ]:
municipios_nulos = ddd[ddd['id_Municipio_IBGE'].isnull()]

In [ ]:
ddd = ddd[ddd['id_Municipio_IBGE'].notnull()]

In [ ]:
ddd['id_Fonte_Dados'] = ddd['id_Fonte_Dados'].astype('int32')

ddd['id_Estado_IBGE'] = ddd['id_Estado_IBGE'].astype('int32')

ddd['id_Municipio_IBGE'] = ddd['id_Municipio_IBGE'].astype('int32')

ddd['DDD'] = ddd['DDD'].astype('int32')

### Inserir dados

In [ ]:
inserir_dados_stg(ddd, 'Stage', 'stg_DDD_Municipio', 'Stage', 'Versiona_stg_DDD_Municipio')

In [ ]:
#Fim da execução do notebook
end_general = datetime.now()

In [ ]:
# Write the DataFrame to a BigQuery table

print("Tempo total notebook " + str(end_general - start_general))